In [3]:
# importaremos las librerias necesarias
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.ml import Pipeline
from pyspark import SparkConf
import os
import shutil
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import desc, split, col,row_number,regexp_replace, lit,to_timestamp,to_date, first, substring, length, col, expr
from pyspark.sql import Window as W
import geopandas as gpd
from shapely import wkt

import pyspark.pandas as ps

In [4]:
spark = SparkSession.builder.master("local[*]").config('spark.driver.extraClassPath', 
 'oracleJDBC/ojdbc8.jar:mssql-jdbc-10.2.1.jre8.jar').getOrCreate()

# lectura y carga de datos

In [6]:
query1 = """select ll.llavePuntoConsumo, NombreRegion, AlturaPuntoConsumo, NombreDepartamentoGeografia, NombreMunicipioGeografia, seccion, grupo
    from DWSRBI_KRONOS.hecho.Llaves ll 
    inner join DWSRBI_KRONOS.Dimension.Geografia g on ll.llaveGeografia = g.LlaveGeografia
	inner join DWSRBI_KRONOS.Dimension.PuntoConsumo pc on ll.llavePuntoConsumo = pc.LlavePuntoConsumo
	inner join [DWSRBI_KRONOS].[Dimension].[ActividadEconomica] an on ll.llaveActividadEconomica = an.LlaveActividadEconomica
    inner join  [DWSRBI_KRONOS].[Dimension].[Zona] zn on ll.llaveZona = zn.LlaveZona """


query2 = """
  SELECT [LlaveConsumoNormalizado]
      ,[LlaveFecha]
      ,[LlavePuntoConsumo]
      ,[csmo_n]
  FROM [DWSRBI_KRONOS].[Hecho].[ConsumoNormalizado]
"""

In [7]:
# diferentes rangos
finalOrderColumns = ['LlavePuntoConsumo',
 '202106',
 '202107',
 '202108',
 '202109',
 '202110',
 '202111',
 '202112',
 '202201',
 '202202',
 '202203',
 '202204',
 '202205',
 'AlturaPuntoConsumo',
 'NombreDepartamentoGeografia',
 'NombreMunicipioGeografia',
 'seccion',
 'grupo',
 'DescripcionAnomalia',
 'anomalias_anuales',
 'NombreRegion',
 'fraude']
finalOrderColumns

['LlavePuntoConsumo',
 '202106',
 '202107',
 '202108',
 '202109',
 '202110',
 '202111',
 '202112',
 '202201',
 '202202',
 '202203',
 '202204',
 '202205',
 'AlturaPuntoConsumo',
 'NombreDepartamentoGeografia',
 'NombreMunicipioGeografia',
 'seccion',
 'grupo',
 'DescripcionAnomalia',
 'anomalias_anuales',
 'NombreRegion',
 'fraude']

In [105]:
finalOrderColumns = ['LlavePuntoConsumo',
 '201906',
 '201907',
 '201908',
 '201909',
 '201910',
 '201911',
 '201912',
 '202001',
 '202002',
 '202003',
 '202004',
 '202005',
 'AlturaPuntoConsumo',
 'NombreDepartamentoGeografia',
 'NombreMunicipioGeografia',
 'seccion',
 'grupo',
 'DescripcionAnomalia',
 'anomalias_anuales',
 'NombreRegion',
 'fraude']
finalOrderColumns

['LlavePuntoConsumo',
 '201906',
 '201907',
 '201908',
 '201909',
 '201910',
 '201911',
 '201912',
 '202001',
 '202002',
 '202003',
 '202004',
 '202005',
 'AlturaPuntoConsumo',
 'NombreDepartamentoGeografia',
 'NombreMunicipioGeografia',
 'seccion',
 'grupo',
 'DescripcionAnomalia',
 'anomalias_anuales',
 'NombreRegion',
 'fraude']

## lectura fraudes

In [8]:
fraudes_df = spark.read \
    .format("jdbc") \
    .option("url", url) \
    .option("dbtable", ' [sgc].[SGCPRO].[trabpend_fr]') \
    .option("user", username) \
    .option("password", password) \
    .option("TrustServerCertificate",True) \
    .option("driver", driver_mssql) \
    .load().select(col('cod_cli'),col('f_detec'),col('nis_rad'))\
    .dropna()
split_col = split(fraudes_df['f_detec'], '-')
fraudes_df = fraudes_df.withColumn("fraud_year",split_col.getItem(0)).withColumn("fraud_month",split_col.getItem(1)).withColumn("fraude",lit(1))
fraudes_df = fraudes_df.filter((fraudes_df.f_detec >= '2021-07-01') & (fraudes_df.f_detec <= '2022-05-31')).dropna().orderBy(col('f_detec').desc()).dropDuplicates(['nis_rad'])

fraudes_df.pandas_api().head()

,cod_cli,f_detec,nis_rad,fraud_year,fraud_month,fraude
0,10003945,2022-01-05,1003945,2022,01,1
1,50000443,2021-12-28,1008358,2021,12,1
2,50966926,2021-12-23,1012303,2021,12,1
3,10020533,2021-11-23,1020533,2021,11,1
4,10023417,2022-04-18,1023417,2022,04,1


In [66]:
fraudes_df.filter(fraudes_df.f_detec <= '2021-06-30').pandas_api().head()

,cod_cli,f_detec,nis_rad,fraud_year,fraud_month,fraude
0,51261528,2021-06-30,5605164,2021,06,1
1,50040561,2021-06-30,5036209,2021,06,1
2,50944419,2021-06-30,5919591,2021,06,1
3,50746263,2021-06-30,5727557,2021,06,1
4,20172223,2021-06-30,2172229,2021,06,1


In [23]:
fraudes_df.filter((fraudes_df.f_detec >= '2021-06-01') & (fraudes_df.f_detec <= '2022-05-31')).count()

12425

In [52]:
fraudes_df.filter(fraudes_df.fraud_year== '2020' ).count()

12690

In [7]:
fraudes_df.count()

28873

In [39]:
fraudes_df.filter(col('nis_rad')==3035443).pandas_api().head(10)

,cod_cli,f_detec,nis_rad,fraud_year,fraud_month,fraude
0,30035436,2022-03-05,3035443,2022,03,1


In [26]:
fraudes_df.groupBy("cod_cli").count().where("count > 1").pandas_api().head()

,cod_cli,count
0,51300975,2
1,51107675,2
2,20231037,2
3,51204073,2
4,51197087,2


## lectura anomalias

In [9]:
#Lectura_dimension_punto_consumo (lr)
anomalias_df = spark.read \
    .format("jdbc") \
    .option("url", url) \
    .option("dbtable", '[DWSRBI_KRONOS].[Hecho].[Anomalia]') \
    .option("user", username) \
    .option("password", password) \
    .option("TrustServerCertificate",True) \
    .option("driver", driver_mssql) \
    .load().filter(col('LlaveFecha')!=0) \
    .select(col('LlavePuntoConsumo').alias('LlavePuntoConsumoAnom'),col('LlaveFecha'),col('DescripcionAnomalia'))\
    .withColumn("ano_mes_anomaly",expr("substring(LlaveFecha, 1, length(LlaveFecha)-2)"))\
    .dropna().filter((col('LlaveFecha') >= '20210601') & (col('LlaveFecha') <= '20220531'))#.filter(col('ano_mes_anomaly')==202205)#.drop(col('LlaveFecha'))


anomalias_df.pandas_api().head()


,LlavePuntoConsumoAnom,LlaveFecha,DescripcionAnomalia,ano_mes_anomaly
0,1000048,20210602,Tapa rota/falta,202106
1,1001407,20210602,Observaciones,202106
2,1001479,20210602,FALTA DE MEDIDOR BTS Y ACOMETIDA,202106
3,1002388,20210602,Observaciones,202106
4,1002494,20210604,FALTA DE MEDIDOR BTS Y ACOMETIDA,202106


In [9]:
anomalias_df.count()

3088257

In [10]:
anomalias_conteo_ano_df = anomalias_df.groupBy("LlavePuntoConsumoAnom").count().where("count > 1")\
.withColumnRenamed('count','anomalias_anuales').withColumnRenamed('LlavePuntoConsumoAnom','LlavePuntoConsumoAnomCount')
anomalias_conteo_ano_df.pandas_api().head()

,LlavePuntoConsumoAnomCount,anomalias_anuales
0,2008677,5
1,2035192,2
2,2031775,2
3,2032791,12
4,2068884,12


In [13]:
anomalias_df.filter(col('LlavePuntoConsumoAnom')==5954564).pandas_api().head(10)

,LlavePuntoConsumoAnom,LlaveFecha,DescripcionAnomalia,ano_mes_anomaly
0,5954564,20191011,Observaciones,201910
1,5954564,20191211,Observaciones,201912


In [30]:
anomalias_df.count()

2523411

## lectura consumos

In [11]:
#Lectura query 2 consumos
consumos_df = spark.read \
    .format("jdbc") \
    .option("url", url) \
    .option("query", query2) \
    .option("user", username) \
    .option("password", password) \
    .option("TrustServerCertificate",True) \
    .option("driver", driver_mssql) \
    .load().dropna().filter(col('LlaveFecha')!=0)

consumos_df.printSchema()

root
 |-- LlaveConsumoNormalizado: integer (nullable = true)
 |-- LlaveFecha: integer (nullable = true)
 |-- LlavePuntoConsumo: integer (nullable = true)
 |-- csmo_n: decimal(15,5) (nullable = true)



In [ ]:
consumos_df.filter(col('LlaveFecha')==)

## lectura datos miscellaneous

In [12]:
#Lectura quer 1


miscellaneous = spark.read \
    .format("jdbc") \
    .option("url", url) \
    .option("query", query1) \
    .option("user", username) \
    .option("password", password) \
    .option("TrustServerCertificate",True) \
    .option("driver", driver_mssql) \
    .load().dropna().withColumnRenamed('llavePuntoConsumo','llavePuntoConsumoMisc')

miscellaneous.pandas_api().head()

,llavePuntoConsumoMisc,NombreRegion,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo
0,2128283,SurOccidente I,62.000000,SUCHITEPEQUEZ,SAN JOSE LA MAQUINA,Residenciales,Residenciales
1,2455402,CentroOccidente II,2508.000000,QUICHE,CHICHICASTENANGO,Residenciales,Residenciales
2,3002794,NorOriente I,457.000000,CHIQUIMULA,CHIQUIMULA,Residenciales,Residenciales
3,5215244,NorOccidente,0.000000,HUEHUETENANGO,SAN ANTONIO HUISTA,Residenciales,Residenciales
4,2264463,CentroOccidente I,2655.000000,SAN MARCOS,SAN ANTONIO SACATEPEQUEZ,Residenciales,Residenciales


In [13]:



miscellaneous_final = miscellaneous.join(anomalias_df, miscellaneous.llavePuntoConsumoMisc == anomalias_df.LlavePuntoConsumoAnom, 'left')\
.drop(col('LlavePuntoConsumoAnom')).fillna('NO REGSITRADO')

w = W.partitionBy(col('llavePuntoConsumoMisc')).orderBy(desc(col('ano_mes_anomaly')))

miscellaneous_final = miscellaneous_final.withColumn("row_number",row_number().over(w)).where(col('row_number')<=1)
miscellaneous_final.pandas_api().head()


,llavePuntoConsumoMisc,NombreRegion,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,LlaveFecha,DescripcionAnomalia,ano_mes_anomaly,row_number
0,1,CentroOccidente I,0.000000,QUETZALTENANGO,SAN MARTIN SACATEPEQUEZ,Residenciales,NO DEFINIDA,NaN,NO REGSITRADO,NO REGSITRADO,1
1,1000005,CentroOccidente II,1807.000000,CHIMALTENANGO,CHIMALTENANGO,Oficinas,SERVICIOS JURIDICOS,NaN,NO REGSITRADO,NO REGSITRADO,1
2,1000007,CentroOccidente II,1809.000000,CHIMALTENANGO,CHIMALTENANGO,Talleres,TALLERES DE REPARACION DE AUTOMOVILES Y BICICL...,20220404.0,Observaciones,202204,1
3,1000013,CentroOccidente II,1814.000000,CHIMALTENANGO,CHIMALTENANGO,Restaurantes,"RESTAURANTES, CAFETERIAS Y COMEDORES",NaN,NO REGSITRADO,NO REGSITRADO,1
4,1000027,CentroOccidente II,1807.000000,CHIMALTENANGO,CHIMALTENANGO,Abarroterías/tiendas,"ABARROTERIAS, TIENDAS DE COMESTIBLES, BANCAS M...",NaN,NO REGSITRADO,NO REGSITRADO,1


In [14]:
from pyspark.sql.functions import when
miscellaneous_final = miscellaneous_final.alias("a").join(anomalias_conteo_ano_df.alias("b"),col("a.llavePuntoConsumoMisc") == col("b.LlavePuntoConsumoAnomCount"),'left')
miscellaneous_final = miscellaneous_final.na.fill(value=0,subset=["anomalias_anuales"])#.withColumn('anomalias_anuales',regexp_replace(miscellaneous_finalx.anomalias_anuales,'NaN',0))
miscellaneous_final.pandas_api().head()

,llavePuntoConsumoMisc,NombreRegion,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,LlaveFecha,DescripcionAnomalia,ano_mes_anomaly,row_number,LlavePuntoConsumoAnomCount,anomalias_anuales
0,1,CentroOccidente I,0.000000,QUETZALTENANGO,SAN MARTIN SACATEPEQUEZ,Residenciales,NO DEFINIDA,NaN,NO REGSITRADO,NO REGSITRADO,1,NaN,0
1,1000005,CentroOccidente II,1807.000000,CHIMALTENANGO,CHIMALTENANGO,Oficinas,SERVICIOS JURIDICOS,NaN,NO REGSITRADO,NO REGSITRADO,1,NaN,0
2,1000007,CentroOccidente II,1809.000000,CHIMALTENANGO,CHIMALTENANGO,Talleres,TALLERES DE REPARACION DE AUTOMOVILES Y BICICL...,20220404.0,Observaciones,202204,1,1000007.0,3
3,1000013,CentroOccidente II,1814.000000,CHIMALTENANGO,CHIMALTENANGO,Restaurantes,"RESTAURANTES, CAFETERIAS Y COMEDORES",NaN,NO REGSITRADO,NO REGSITRADO,1,NaN,0
4,1000027,CentroOccidente II,1807.000000,CHIMALTENANGO,CHIMALTENANGO,Abarroterías/tiendas,"ABARROTERIAS, TIENDAS DE COMESTIBLES, BANCAS M...",NaN,NO REGSITRADO,NO REGSITRADO,1,NaN,0


In [34]:
miscellaneous.filter(col('llavePuntoConsumoMisc')==3035443).show()

+---------------------+------------+------------------+---------------------------+------------------------+-------------+-------------+
|llavePuntoConsumoMisc|NombreRegion|AlturaPuntoConsumo|NombreDepartamentoGeografia|NombreMunicipioGeografia|      seccion|        grupo|
+---------------------+------------+------------------+---------------------------+------------------------+-------------+-------------+
|              3035443|NorOriente I|          0.000000|                     IZABAL|                 MORALES|Residenciales|Residenciales|
+---------------------+------------+------------------+---------------------------+------------------------+-------------+-------------+



In [83]:
miscellaneous.count()

2240017

# organizar consumos y anomalias respecto a la utíma lectura

In [15]:
W = W.partitionBy("LlavePuntoConsumo").orderBy(col('LlavePuntoConsumo'),col('LlaveFecha').desc())

#.dropDuplicates(['LlaveFecha','LlavePuntoConsumo'])
result_w = consumos_df.dropDuplicates(['LlaveFecha','LlavePuntoConsumo']).withColumn("row_number",row_number().over(W)).where(col('row_number')<=20)#45

#result_w = result_w.withColumn("count",F.count(col('LlavePuntoConsumo')).over(W)).where(col('count')==12)

result_w = result_w.withColumn("ano_mes_lectura",expr("substring(LlaveFecha, 1, length(LlaveFecha)-2)")).cache()


#result_w.pandas_api().head()


In [ ]:
#W = W.partitionBy("LlavePuntoConsumo").orderBy(col('LlavePuntoConsumo'),col('LlaveFecha').desc())

#.dropDuplicates(['LlaveFecha','LlavePuntoConsumo'])
#result_w = consumos_df.dropDuplicates(['LlaveFecha','LlavePuntoConsumo']).withColumn("row_number",row_number().over(W))\
#.where((col('row_number')<=45) & (col('row_number')>10))

#result_w = result_w.withColumn("count",F.count(col('LlavePuntoConsumo')).over(W)).where(col('count')==12)

#result_w = result_w.withColumn("ano_mes_lectura",expr("substring(LlaveFecha, 1, length(LlaveFecha)-2)")).cache()


#result_w.pandas_api().head(50)


In [ ]:
result_w.pandas_api().head(100)

In [13]:

#result_join_anomaly = result_w.join(anomalias_df, (result_w.LlavePuntoConsumo == anomalias_df.LlavePuntoConsumoAnom) \
 #                                   & (result_w.ano_mes_lectura == anomalias_df.ano_mes_anomaly), 'left').fillna('NO REGISTRADO')#.drop(miscellaneous.llavePuntoConsumo)



#W = W.partitionBy("LlavePuntoConsumo").orderBy(col('LlavePuntoConsumo'),col('LlaveFecha').desc())

#last_anomaly_reading = result_join_anomaly.withColumn("row_number",row_number().over(W)).where(col('row_number') == 1)\
#.select(col('LlavePuntoConsumoAnom'),col('DescripcionAnomalia'))

#last_anomaly_reading.pandas_api().head()
#result_join_anomaly.pandas_api().head()

In [ ]:
#result_join_anomaly.filter(col('ano_mes_lectura')==202109).pandas_api().head()

In [19]:
#result_join_anomaly.count()

43157211

In [16]:
#filter last 13 months
from pyspark.sql.functions import coalesce, when

resultSeriescache = result_w.groupBy("LlavePuntoConsumo").pivot("ano_mes_lectura").agg(first("csmo_n")).cache()
last12Months=resultSeriescache.columns[-15:-3]#-27,-15
#last12Months=resultSeriescache.columns[-29:-17]#-27,-15
last12Months.insert(0,'LlavePuntoConsumo')

resultSeries = resultSeriescache.select(last12Months).dropna()#fillna(1,subset=['202206']).dropna()
resultSeries.pandas_api().head()


,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205
0,1000149,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,1000190,237.90000,222.73000,222.73000,212.92000,226.10000,229.08000,203.48000,200.56000,236.47000,247.53000,232.18000,271.72000
2,1000636,135.79000,153.06000,162.88000,171.35000,180.47000,181.55000,148.94000,169.19000,176.61000,179.35000,144.99000,151.07000
3,1001043,105.37000,97.14000,85.36000,83.14000,82.13000,85.55000,83.91000,75.09000,68.68000,79.71000,88.31000,125.86000
4,1001139,134.70000,131.48000,156.99000,122.68000,150.06000,109.31000,102.79000,120.72000,132.46000,129.01000,158.95000,145.79000


In [ ]:

5036209 
5919591 
5727557 
2172229 

In [99]:
resultSeries.filter(col('LlavePuntoConsumo')==5727557).pandas_api().head(20)

,LlavePuntoConsumo,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005
0,5727557,85.17000,88.31000,443.49000,569.53000,520.89000,199.18000,220.26000,214.88000,235.22000,249.42000,255.69000,285.16000


In [16]:
resultSeries.count()

2033052

In [94]:
resultSeries.count()

1818351

In [ ]:
resultSeries.columns

In [34]:
resultSeries.groupBy("LlavePuntoConsumo").count().where("count > 1").pandas_api().head()

,LlavePuntoConsumo,count


# casos NO fraudulentos

In [23]:
fraude1 = spark.read.csv('fraudes_csv_202006-202105/part-00000-8487a199-c9f0-4495-8039-5d9b6958d0f7-c000.csv', header = True).\
withColumnRenamed('LlavePuntoConsumo','LlavePuntoConsumo1')
fraude1.pandas_api().head()

,LlavePuntoConsumo1,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,anomalias_anuales,NombreRegion,fraude
0,1002467,122.72000,111.53000,113.11000,97.14000,102.40000,95.17000,121.67000,125.47000,133.44000,114.80000,124.71000,123.69000,1784.000000,CHIMALTENANGO,CHIMALTENANGO,Residenciales,Residenciales,NO REGSITRADO,0,CentroOccidente II,1
1,1017184,0.00000,3.15000,10.46000,114.57000,1.01000,1.90000,1.05000,0.95000,0.00000,0.00000,0.00000,0.00000,1779.000000,CHIMALTENANGO,SAN MARTIN JILOTEPEQUE,Residenciales,Residenciales,Presunto Fraude,0,CentroOccidente II,1
2,2003713,93.28000,76.57000,95.05000,61.85000,75.55000,81.74000,73.87000,73.59000,65.59000,83.14000,143.25000,93.28000,2263.000000,HUEHUETENANGO,SOLOMA,Residenciales,Residenciales,Observaciones,4,NorOccidente,1
3,2017769,0.00000,0.00000,0.00000,0.00000,1.01000,0.00000,0.00000,0.00000,0.00000,0.00000,138.35000,156.01000,1432.000000,HUEHUETENANGO,UNION CANTINIL,Residenciales,Residenciales,Observaciones,0,NorOccidente,1
4,2018282,5.07000,5.24000,4.75000,4.06000,3.92000,3.80000,10.86000,18.06000,18.64000,6.87000,7.10000,8.11000,2634.000000,HUEHUETENANGO,SAN JUAN IXCOY,Residenciales,Residenciales,Presunto Fraude,3,NorOccidente,1


In [18]:
fraude1.count()

2871

In [24]:
fraude2 = spark.read.csv('fraudes_csv_202106-202205/part-00000-6395085e-10fa-4529-99e5-337a238d3fcd-c000.csv',header = True).\
withColumnRenamed('LlavePuntoConsumo1','LlavePuntoConsumo2')

In [20]:
fraude2.count()

8540

In [25]:
fraude3 = spark.read.csv('fraudes_csv_201906-202005/part-00000-e43b5b1e-0ba2-482d-9a81-fcd9d4de2fed-c000.csv',header = True).\
withColumnRenamed('LlavePuntoConsumo1','LlavePuntoConsumo3')

In [26]:
fraude3.count()

7708

In [27]:
g = fraude1.union(fraude2).union(fraude3)
g.pandas_api().head()

,LlavePuntoConsumo1,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,anomalias_anuales,NombreRegion,fraude
0,1002467,122.72000,111.53000,113.11000,97.14000,102.40000,95.17000,121.67000,125.47000,133.44000,114.80000,124.71000,123.69000,1784.000000,CHIMALTENANGO,CHIMALTENANGO,Residenciales,Residenciales,NO REGSITRADO,0,CentroOccidente II,1
1,1017184,0.00000,3.15000,10.46000,114.57000,1.01000,1.90000,1.05000,0.95000,0.00000,0.00000,0.00000,0.00000,1779.000000,CHIMALTENANGO,SAN MARTIN JILOTEPEQUE,Residenciales,Residenciales,Presunto Fraude,0,CentroOccidente II,1
2,2003713,93.28000,76.57000,95.05000,61.85000,75.55000,81.74000,73.87000,73.59000,65.59000,83.14000,143.25000,93.28000,2263.000000,HUEHUETENANGO,SOLOMA,Residenciales,Residenciales,Observaciones,4,NorOccidente,1
3,2017769,0.00000,0.00000,0.00000,0.00000,1.01000,0.00000,0.00000,0.00000,0.00000,0.00000,138.35000,156.01000,1432.000000,HUEHUETENANGO,UNION CANTINIL,Residenciales,Residenciales,Observaciones,0,NorOccidente,1
4,2018282,5.07000,5.24000,4.75000,4.06000,3.92000,3.80000,10.86000,18.06000,18.64000,6.87000,7.10000,8.11000,2634.000000,HUEHUETENANGO,SAN JUAN IXCOY,Residenciales,Residenciales,Presunto Fraude,3,NorOccidente,1


In [28]:
df_no_fraud = resultSeries.join(g, col("LlavePuntoConsumo")==col("LlavePuntoConsumo1"), "left_anti")
df_no_fraud.pandas_api().head()

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205
0,1000149,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,1000190,237.90000,222.73000,222.73000,212.92000,226.10000,229.08000,203.48000,200.56000,236.47000,247.53000,232.18000,271.72000
2,1000636,135.79000,153.06000,162.88000,171.35000,180.47000,181.55000,148.94000,169.19000,176.61000,179.35000,144.99000,151.07000
3,1001043,105.37000,97.14000,85.36000,83.14000,82.13000,85.55000,83.91000,75.09000,68.68000,79.71000,88.31000,125.86000
4,1001139,134.70000,131.48000,156.99000,122.68000,150.06000,109.31000,102.79000,120.72000,132.46000,129.01000,158.95000,145.79000


In [29]:
df_no_fraud.count()

2015219

In [49]:
miscellaneous_final.columns

['llavePuntoConsumoMisc',
 'NombreRegion',
 'AlturaPuntoConsumo',
 'NombreDepartamentoGeografia',
 'NombreMunicipioGeografia',
 'seccion',
 'grupo',
 'LlaveFecha',
 'DescripcionAnomalia',
 'ano_mes_anomaly',
 'LlavePuntoConsumoAnomCount',
 'anomalias_anuales']

In [ ]:

#result_join_anomaly = result_w.join(anomalias_df, (result_w.LlavePuntoConsumo == anomalias_df.LlavePuntoConsumoAnom) \
 #                                   & (result_w.ano_mes_lectura == anomalias_df.ano_mes_anomaly), 'left').fillna('NO REGISTRADO')#.drop(miscellaneous.llavePuntoConsumo)



In [30]:
df_final_no_fraud = df_no_fraud.join(miscellaneous_final,col('llavePuntoConsumoMisc') ==  col('LlavePuntoConsumo'),"inner")\
                    .drop(col('llavePuntoConsumoMisc')).drop('ano_mes_anomaly').drop('LlaveFecha').withColumn("fraude",lit(0)).select(finalOrderColumns)#\
                    #.limit(30000)
df_final_no_fraud.pandas_api().head()

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,anomalias_anuales,NombreRegion,fraude
0,1000149,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1798.000000,CHIMALTENANGO,CHIMALTENANGO,Residenciales,Residenciales,Observaciones,3,CentroOccidente II,0
1,1000190,237.90000,222.73000,222.73000,212.92000,226.10000,229.08000,203.48000,200.56000,236.47000,247.53000,232.18000,271.72000,1807.000000,CHIMALTENANGO,CHIMALTENANGO,Locales comerciales,COMERCIO POR MAYOR DE GAS EN CILINDROS,NO REGSITRADO,0,CentroOccidente II,0
2,1000636,135.79000,153.06000,162.88000,171.35000,180.47000,181.55000,148.94000,169.19000,176.61000,179.35000,144.99000,151.07000,1792.000000,CHIMALTENANGO,CHIMALTENANGO,Oficinas,FINANCIERAS,NO REGSITRADO,0,CentroOccidente II,0
3,1001043,105.37000,97.14000,85.36000,83.14000,82.13000,85.55000,83.91000,75.09000,68.68000,79.71000,88.31000,125.86000,1812.000000,CHIMALTENANGO,CHIMALTENANGO,Residenciales,Residenciales,NO REGSITRADO,0,CentroOccidente II,0
4,1001139,134.70000,131.48000,156.99000,122.68000,150.06000,109.31000,102.79000,120.72000,132.46000,129.01000,158.95000,145.79000,1807.000000,CHIMALTENANGO,CHIMALTENANGO,Residenciales,Residenciales,NO REGSITRADO,0,CentroOccidente II,0


In [32]:
df_final_no_fraud.count()

1990560

In [ ]:
df_final_no_fraud.filter(col('NombreDepartamentoGeografia')=='SOLOLA').show()

In [31]:
df_final_no_fraud.printSchema()

root
 |-- LlavePuntoConsumo: integer (nullable = true)
 |-- 202106: decimal(15,5) (nullable = true)
 |-- 202107: decimal(15,5) (nullable = true)
 |-- 202108: decimal(15,5) (nullable = true)
 |-- 202109: decimal(15,5) (nullable = true)
 |-- 202110: decimal(15,5) (nullable = true)
 |-- 202111: decimal(15,5) (nullable = true)
 |-- 202112: decimal(15,5) (nullable = true)
 |-- 202201: decimal(15,5) (nullable = true)
 |-- 202202: decimal(15,5) (nullable = true)
 |-- 202203: decimal(15,5) (nullable = true)
 |-- 202204: decimal(15,5) (nullable = true)
 |-- 202205: decimal(15,5) (nullable = true)
 |-- AlturaPuntoConsumo: string (nullable = false)
 |-- NombreDepartamentoGeografia: string (nullable = false)
 |-- NombreMunicipioGeografia: string (nullable = false)
 |-- seccion: string (nullable = false)
 |-- grupo: string (nullable = false)
 |-- DescripcionAnomalia: string (nullable = false)
 |-- anomalias_anuales: long (nullable = true)
 |-- NombreRegion: string (nullable = false)
 |-- fraude: in

# casos fraudulentos

In [103]:
df_fraud = resultSeries.join(fraudes_df,col("LlavePuntoConsumo")==col("nis_rad"),"inner")
df_fraud.pandas_api().head()

,LlavePuntoConsumo,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,cod_cli,f_detec,nis_rad,fraud_year,fraud_month,fraude
0,5856681,96.32000,90.20000,106.46000,86.34000,68.18000,66.54000,98.85000,90.27000,75.09000,91.25000,83.65000,139.92000,50863087,2020-05-14,5856681,2020,05,1
1,3186632,82.42000,98.59000,80.79000,73.00000,87.19000,81.74000,94.19000,63.01000,85.55000,88.10000,150.18000,89.22000,30186570,2020-01-22,3186632,2020,01,1
2,3247490,41.82000,113.28000,142.27000,20.91000,75.52000,53.23000,95.60000,15.70000,232.88000,43.00000,72.24000,128.76000,30247415,2020-06-01,3247490,2020,06,1
3,5299669,81.74000,26.22000,26.61000,75.03000,28.39000,13.31000,59.85000,21.73000,60.83000,31.47000,29.44000,190.35000,50309072,2020-04-24,5299669,2020,04,1
4,2164368,94.10000,41.95000,46.12000,55.93000,37.76000,48.48000,47.65000,64.76000,47.65000,105.44000,138.90000,162.88000,20164362,2020-02-10,2164368,2020,02,1


In [26]:
df_fraud.count()
# [sgc].[SGCPRO].[trabpend_fr] entre 2021-06 y 2022-05

9416

In [169]:
miscellaneous.count()

#ll.llavePuntoConsumo,AlturaPuntoConsumo, NombreDepartamentoGeografia, NombreMunicipioGeografia, seccion, grupo
   # de DWSRBI_KRONOS.Dimension.Geografia ,DWSRBI_KRONOS.Dimension.PuntoConsumo 
	# y [DWSRBI_KRONOS].[Dimension].[ActividadEconomica] 

2216257

In [168]:
miscellaneous_final.count()
#igual que miscellaneous pero agregando las anomalias de lectura, se pierden 1.959.392 de datos

256865

In [170]:
anomalias_df.count() #cantidad de anomalias para el mes 05 del año 2022

1488053

In [106]:
df_final_fraud = df_fraud.join(miscellaneous_final,col('llavePuntoConsumoMisc') ==  col('LlavePuntoConsumo'),"inner").select(finalOrderColumns)#\.drop(col('llavePuntoConsumoMisc'))
df_final_fraud.pandas_api().head()


,LlavePuntoConsumo,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,anomalias_anuales,NombreRegion,fraude
0,1004403,26.61000,25.17000,25.66000,15.21000,10.49000,20.28000,36.71000,0.00000,0.00000,47.20000,47.65000,47.53000,1842.000000,CHIMALTENANGO,CHIMALTENANGO,Residenciales,Residenciales,Observaciones,0,CentroOccidente II,1
1,1007766,157.79000,71.32000,62.80000,36.12000,85.17000,44.05000,37.76000,17.66000,47.10000,58.74000,110.61000,84.96000,2114.000000,CHIMALTENANGO,COMALAPA,Residenciales,Residenciales,NO REGSITRADO,0,CentroOccidente II,1
2,2000310,67423.61000,66965.73000,76295.14000,78069.44000,59882.81000,17743.06000,5322.92000,30907.26000,27532.33000,38258.46000,27532.33000,32624.33000,0.000000,SUCHITEPEQUEZ,PATULUL,Servicios Agropecuarios,AVICULTURA,Cierre Forzado / Sin Lectura,0,SurOccidente I,1
3,2003848,94.10000,118.52000,110.51000,101.71000,113.56000,101.71000,115.78000,122.75000,97.90000,119.57000,106.95000,91.25000,0.000000,HUEHUETENANGO,SOLOMA,Residenciales,Residenciales,Conexion directa con medidor,11,NorOccidente,1
4,2004018,67.49000,75.52000,69.96000,68.44000,81.11000,71.29000,78.49000,86.90000,68.44000,81.81000,69.66000,76.04000,2306.000000,HUEHUETENANGO,SOLOMA,Residenciales,Residenciales,Conexion directa con medidor,12,NorOccidente,1


In [28]:
df_final_fraud.filter(col('LlavePuntoConsumo')==3035443).pandas_api().head()#Med. s/precinto en bornera

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,anomalias_anuales,NombreRegion,fraude
0,3035443,0.00000,0.00000,0.00000,175.40000,175.40000,163.49000,176.21000,119.77000,143.25000,135.30000,122.65000,166.77000,0.000000,IZABAL,MORALES,Residenciales,Residenciales,Presunto Fraude,9,NorOriente I,1
1,3035443,0.00000,0.00000,0.00000,175.40000,175.40000,163.49000,176.21000,119.77000,143.25000,135.30000,122.65000,166.77000,0.000000,IZABAL,MORALES,Residenciales,Residenciales,Presunto Fraude,9,NorOriente I,1
2,3035443,0.00000,0.00000,0.00000,175.40000,175.40000,163.49000,176.21000,119.77000,143.25000,135.30000,122.65000,166.77000,0.000000,IZABAL,MORALES,Residenciales,Residenciales,Presunto Fraude,9,NorOriente I,1
3,3035443,0.00000,0.00000,0.00000,175.40000,175.40000,163.49000,176.21000,119.77000,143.25000,135.30000,122.65000,166.77000,0.000000,IZABAL,MORALES,Residenciales,Residenciales,Presunto Fraude,9,NorOriente I,1


In [107]:
df_final_fraud.groupby(col('LlavePuntoConsumo')).count().filter(col('count')>1).show()

+-----------------+-----+
|LlavePuntoConsumo|count|
+-----------------+-----+
+-----------------+-----+



In [34]:
anomalias_df.printSchema()

root
 |-- LlavePuntoConsumoAnom: integer (nullable = true)
 |-- DescripcionAnomalia: string (nullable = true)
 |-- ano_mes_anomaly: string (nullable = true)



In [108]:
df_final_fraud.count()

7708

In [97]:
#df_final_fraud.filter(col('NombreDepartamentoGeografia')=='SOLOLA').show()

+-----------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+---------------------------+------------------------+--------------------+--------------------+--------------------+------+
|LlavePuntoConsumo|   202106|   202107|   202108|   202109|   202110|   202111|   202112|   202201|   202202|   202203|   202204|   202205|AlturaPuntoConsumo|NombreDepartamentoGeografia|NombreMunicipioGeografia|             seccion|               grupo| DescripcionAnomalia|fraude|
+-----------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+---------------------------+------------------------+--------------------+--------------------+--------------------+------+
|          2097768| 16.78000| 15.21000| 22.57000| 29.44000|  9.51000|  0.00000| 23.07000| 21.86000| 18.25000|  0.00000|  0.00000|  0.00000|       1141.000

# guardar datos


In [109]:
df_final_fraud.coalesce(1).write.csv("fraudes_csv_201906-202005", header = True, mode='overwrite')

In [33]:
df_final_no_fraud.coalesce(1).write.csv("no_fraudes_todos_clientes_regiones_csv_v4", header = True, mode='overwrite')

In [30]:
x = pd.read_csv("fraudes_csv/part-00000-d373a665-bc54-459f-bafe-3fe470516547-c000.csv").drop_duplicates('LlavePuntoConsumo')


y = pd.read_csv("no_fraudes_csv/part-00000-31d6be73-a202-40bc-9245-fa77521b7e70-c000.csv")
y = y.sample(frac=1)[:27000]

In [32]:
x['LlavePuntoConsumo'].nunique() - x['LlavePuntoConsumo'].count()

0

In [59]:
y.head(100)

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,fraude
1704685,5560329,6.52,8.83,7.85,7.85,8.83,7.10,8.39,6.65,6.08,7.10,7.10,8.11,0.0,QUICHE,SAN JUAN COTZAL,Residenciales,Residenciales,NO REGSITRADO,0
430645,6396816,6.52,7.85,4.06,4.91,5.89,4.91,5.24,5.89,5.89,6.08,4.06,6.29,0.0,TOTONICAPAN,SANTA MARIA CHIQUIMULA,Residenciales,Residenciales,Observaciones,0
386947,6207179,78.66,73.19,82.86,76.04,80.46,87.05,84.96,-64.64,6.87,76.04,103.02,38.02,0.0,HUEHUETENANGO,BARILLAS,Residenciales,Residenciales,Fuerza Mayor,0
720263,5495656,42.37,52.00,45.63,44.15,50.04,51.02,54.54,49.06,46.12,51.71,50.69,51.39,0.0,SAN MARCOS,RIO BLANCO,Residenciales,Residenciales,NO REGSITRADO,0
963841,6631621,35.85,40.87,33.46,42.19,66.72,33.46,36.50,91.25,42.58,45.13,47.20,50.69,0.0,ESCUINTLA,NUEVA CONCEPCION,Residenciales,Residenciales,NO REGSITRADO,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1549840,2472416,30.42,27.47,27.38,26.61,27.47,25.17,24.12,22.81,23.32,25.51,31.40,29.33,1111.0,QUICHE,SAN ANDRES SAJCABAJA,Residenciales,Residenciales,NO REGSITRADO,0
1876657,6336905,28.24,28.52,25.35,32.38,28.45,34.47,31.43,42.19,44.15,38.53,42.58,50.34,0.0,SOLOLA,SAN ANDRES SEMETABAJ,Residenciales,Residenciales,NO REGSITRADO,0
447625,6477986,93.35,96.32,73.59,98.12,107.47,105.97,104.89,104.99,96.16,10.14,4.35,3.04,0.0,CHIQUIMULA,ESQUIPULAS,Residenciales,Residenciales,NO REGSITRADO,0
116982,3075547,79.30,75.03,62.86,52.00,48.08,46.12,45.10,49.06,41.21,53.74,68.18,72.61,144.0,ZACAPA,GUALAN,Residenciales,Residenciales,NO REGSITRADO,0


In [33]:
z = pd.concat([x,y])

In [34]:
z['LlavePuntoConsumo'].nunique() - z['LlavePuntoConsumo'].count()

0

In [54]:
len(x)

17881

In [36]:
df_completa = z.sample(frac=1)

In [64]:
df_completa.head()

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,fraude
1566672,3052515,129.27,124.52,101.39,120.65,132.46,122.65,95.45,94.02,84.15,92.30,116.76,115.37,54.0,IZABAL,MORALES,Residenciales,Residenciales,NO REGSITRADO,0
1281280,5880480,71.32,71.63,59.82,67.49,51.02,54.54,51.02,53.74,70.97,72.61,85.36,94.51,162.0,SUCHITEPEQUEZ,RIO BRAVO,Residenciales,Residenciales,NO REGSITRADO,0
8960,6631486,3.15,1.01,1.96,1.96,3.80,4.20,2.10,0.98,3.92,2.03,1.90,1.09,0.0,JALAPA,SAN PEDRO PINULA,Residenciales,Residenciales,NO REGSITRADO,1
65731,2320981,11.95,11.41,12.17,13.18,15.70,13.74,13.64,13.31,12.76,13.18,14.19,17.24,2609.0,QUETZALTENANGO,CONCEPCION CHIQUIRICHAPA,Residenciales,Residenciales,NO REGSITRADO,0
16559,5839450,114.06,66.72,67.70,67.93,58.93,62.86,52.44,52.28,61.85,38.53,41.57,38.53,2095.0,TOTONICAPAN,SANTA MARIA CHIQUIMULA,Residenciales,Residenciales,NO REGSITRADO,1


In [37]:
df_completa.to_csv('data_final_entrenamiento_202106_202205.csv',header = True, index=False)

In [66]:
df_completa.columns

Index(['LlavePuntoConsumo', '202106', '202107', '202108', '202109', '202110',
       '202111', '202112', '202201', '202202', '202203', '202204', '202205',
       'AlturaPuntoConsumo', 'NombreDepartamentoGeografia',
       'NombreMunicipioGeografia', 'seccion', 'grupo', 'DescripcionAnomalia',
       'fraude'],
      dtype='object')

In [55]:
x = pd.read_csv("fraudes_csv/part-00000-d373a665-bc54-459f-bafe-3fe470516547-c000.csv")


y = pd.read_csv("no_fraudes_csv/part-00000-31d6be73-a202-40bc-9245-fa77521b7e70-c000.csv")

In [45]:
x['LlavePuntoConsumo'].nunique() - x['LlavePuntoConsumo'].count()

-1400

In [36]:
x = x.drop_duplicates('LlavePuntoConsumo')

In [39]:
x[x['LlavePuntoConsumo']==5179865]

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,fraude
1173,5179865,137.96,137.37,150.12,105.44,80.1,59.88,162.57,142.58,147.18,149.99,155.03,154.18,0.0,RETALHULEU,CHAMPERICO,Residenciales,Residenciales,Observaciones,1


In [60]:
y.head()

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,fraude
0,1000005,234.64,184.46,218.80,190.61,162.22,183.45,181.45,181.55,185.44,189.84,180.47,185.54,1807.0,CHIMALTENANGO,CHIMALTENANGO,Oficinas,SERVICIOS JURIDICOS,NO REGSITRADO,0
1,1000007,1.09,0.98,0.00,1.01,0.00,0.00,0.00,0.95,0.98,2.10,1.96,1.05,1809.0,CHIMALTENANGO,CHIMALTENANGO,Talleres,TALLERES DE REPARACION DE AUTOMOVILES Y BICICL...,NO REGSITRADO,0
2,1000013,583.35,553.39,467.04,408.60,399.47,429.64,416.39,422.98,322.81,309.41,304.17,249.63,1814.0,CHIMALTENANGO,CHIMALTENANGO,Restaurantes,"RESTAURANTES, CAFETERIAS Y COMEDORES",NO REGSITRADO,0
3,1000027,348.71,337.53,319.87,334.58,327.49,308.92,325.14,315.57,260.99,284.24,286.51,337.73,1807.0,CHIMALTENANGO,CHIMALTENANGO,Abarroterías/tiendas,"ABARROTERIAS, TIENDAS DE COMESTIBLES, BANCAS M...",NO REGSITRADO,0
4,1000034,159.69,170.73,124.61,59.82,71.99,101.71,206.62,203.41,92.23,74.47,81.44,72.37,1809.0,CHIMALTENANGO,CHIMALTENANGO,Residenciales,Residenciales,NO REGSITRADO,0


In [40]:
z = pd.concat([x,y])
df_completa = z.sample(frac=1)

In [58]:
y.to_csv('data_final_todos_los_clientes.csv',header = True, index=False)

In [72]:
len(df_completa)

1993529

### Regiones especiales

In [26]:
x.columns

Index(['LlavePuntoConsumo', '202006', '202007', '202008', '202009', '202010',
       '202011', '202012', '202101', '202102', '202103', '202104', '202105',
       'AlturaPuntoConsumo', 'NombreDepartamentoGeografia',
       'NombreMunicipioGeografia', 'seccion', 'grupo', 'DescripcionAnomalia',
       'anomalias_anuales', 'NombreRegion', 'fraude'],
      dtype='object')

In [34]:
df1 = pd.read_csv("fraudes_csv_202006-202105/part-00000-8487a199-c9f0-4495-8039-5d9b6958d0f7-c000.csv")


df2 = pd.read_csv("fraudes_csv_202106-202205/part-00000-6395085e-10fa-4529-99e5-337a238d3fcd-c000.csv")

df3 = pd.read_csv("fraudes_csv_201906-202005/part-00000-e43b5b1e-0ba2-482d-9a81-fcd9d4de2fed-c000.csv")

df1.columns = df2.columns
df3.columns = df2.columns

z = pd.concat([df1,df2,df3])
z.head()

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,...,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,anomalias_anuales,NombreRegion,fraude
0,1002467,122.72,111.53,113.11,97.14,102.40,95.17,121.67,125.47,133.44,...,123.69,1784.0,CHIMALTENANGO,CHIMALTENANGO,Residenciales,Residenciales,NO REGSITRADO,0,CentroOccidente II,1
1,1017184,0.00,3.15,10.46,114.57,1.01,1.90,1.05,0.95,0.00,...,0.00,1779.0,CHIMALTENANGO,SAN MARTIN JILOTEPEQUE,Residenciales,Residenciales,Presunto Fraude,0,CentroOccidente II,1
2,2003713,93.28,76.57,95.05,61.85,75.55,81.74,73.87,73.59,65.59,...,93.28,2263.0,HUEHUETENANGO,SOLOMA,Residenciales,Residenciales,Observaciones,4,NorOccidente,1
3,2017769,0.00,0.00,0.00,0.00,1.01,0.00,0.00,0.00,0.00,...,156.01,1432.0,HUEHUETENANGO,UNION CANTINIL,Residenciales,Residenciales,Observaciones,0,NorOccidente,1
4,2018282,5.07,5.24,4.75,4.06,3.92,3.80,10.86,18.06,18.64,...,8.11,2634.0,HUEHUETENANGO,SAN JUAN IXCOY,Residenciales,Residenciales,Presunto Fraude,3,NorOccidente,1


In [37]:
len(z)

2145

In [36]:
regiones = ['SurOriente', 'CentroOccidente II','NorOriente II']

z = z[z['NombreRegion'].isin(regiones)] 
z.head()

,LlavePuntoConsumo,202106,202107,202108,202109,202110,202111,202112,202201,202202,...,202205,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,anomalias_anuales,NombreRegion,fraude
0,1002467,122.72,111.53,113.11,97.14,102.40,95.17,121.67,125.47,133.44,...,123.69,1784.0,CHIMALTENANGO,CHIMALTENANGO,Residenciales,Residenciales,NO REGSITRADO,0,CentroOccidente II,1
1,1017184,0.00,3.15,10.46,114.57,1.01,1.90,1.05,0.95,0.00,...,0.00,1779.0,CHIMALTENANGO,SAN MARTIN JILOTEPEQUE,Residenciales,Residenciales,Presunto Fraude,0,CentroOccidente II,1
149,2405770,111.18,147.18,161.90,236.47,344.72,323.79,90.20,81.74,122.65,...,245.36,2530.0,SOLOLA,SANTA LUCIA UTATLAN,Residenciales,Residenciales,NO REGSITRADO,0,CentroOccidente II,1
150,2442128,58.87,62.93,94.19,294.35,227.11,207.21,205.57,212.92,238.58,...,208.99,2033.0,QUICHE,SANTA CRUZ DEL QUICHE,Residenciales,Residenciales,NO REGSITRADO,0,CentroOccidente II,1
231,3161851,25.51,13.64,2.94,11.41,11.54,13.31,23.07,27.57,39.25,...,41.21,921.0,SANTA ROSA,CUILAPA,Residenciales,Residenciales,Tapa rota/falta,0,SurOriente,1


In [38]:
no_fraude = pd.read_csv("no_fraudes_regiones_csv/part-00000-ef3c76fe-476a-40db-b783-214fa4633edb-c000.csv")
no_fraude = no_fraude[no_fraude['NombreRegion'].isin(regiones)].sample(frac=1)[:2500]
no_fraude.columns = df2.columns

In [67]:
no_fraude.head()

,LlavePuntoConsumo,202006,202007,202008,202009,202010,202011,202012,202101,202102,...,202105,AlturaPuntoConsumo,NombreDepartamentoGeografia,NombreMunicipioGeografia,seccion,grupo,DescripcionAnomalia,anomalias_anuales,NombreRegion,fraude
1226578,6278189,20.64,19.01,21.29,11.77,17.66,18.25,23.32,24.53,18.64,...,12.59,0.0,JUTIAPA,JUTIAPA,Residenciales,Residenciales,Observaciones,0,NorOriente II,0
634181,5608896,19.93,16.22,34.47,44.15,47.10,43.17,31.47,27.47,16.68,...,12.17,2399.0,SOLOLA,SOLOLA,Residenciales,Residenciales,NO REGSITRADO,0,CentroOccidente II,0
1197847,6554056,15.21,16.16,15.70,19.62,32.44,32.44,33.46,35.32,41.57,...,40.56,0.0,SANTA ROSA,PUEBLO NUEVO VI?AS,Residenciales,Residenciales,NO REGSITRADO,0,SurOriente,0
376165,6089131,466.03,537.99,550.54,514.23,603.43,613.58,668.19,629.63,618.47,...,571.40,1279.0,JUTIAPA,MOYUTA,Abarroterías/tiendas,ABARROTERIA/MINI MARKET (CON 2 O MAS CAMARAS),NO REGSITRADO,0,SurOriente,0
597834,6396418,65.03,63.78,63.88,57.98,32.38,3.15,0.00,0.00,0.00,...,0.00,0.0,JUTIAPA,SAN JOSE ACATEMPA,Residenciales,Residenciales,NO REGSITRADO,0,NorOriente II,0


In [39]:
z = pd.concat([z,no_fraude])

In [116]:
len(z)

4645

In [ ]:
pd.concat(g for _, g in z.groupby("LlavePuntoConsumo") if len(g) > 1)

In [57]:
from pandas_profiling import ProfileReport
#profile = ProfileReport(z, title="Pandas Profiling Report")

In [117]:
z.profile_report().to_file("reporte_regiones_especiales.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [40]:
z.to_csv('regiones_especiales.csv',header = True, index=False)